# 06-06

6000668- Transition matrices + continous
    - Remove Constant Features
    - BoxCox
    - Center and Scaling
    - Euclidean Distance
- Categorical One Hot Encoded Features
    - Jaccard
- Weighted rank result of euclidean and jackard

In [89]:
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt 
import math
from ipynb.fs.full.Utils import createKDeplot, loadDataFrames, dropFeaturesWithNoVariance, loadCoverGroups
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import stats
from ipynb.fs.full.Pipeline import removeConstantFeatures, applyBoxCoxAllFeatures, centerAndScale, applyEuclideanDistance, getIndexAndVector, applyJaccard

In [112]:
cover = loadCoverGroups()
listCovers = cover[17]
originalSongId = listCovers[0]
coverSongId = listCovers[1]

In [113]:
cover

[[180849,
  6001027,
  1686718,
  5000013,
  5000001,
  5000007,
  5000008,
  5000023,
  5000017,
  5000019,
  5000020,
  5000018,
  5000011,
  5000012],
 [6001971, 6001984],
 [6002492, 6002491],
 [6000699, 6001165, 6000182],
 [6000104, 6000273],
 [6000768, 6000784],
 [6001931, 6001748],
 [6002043, 6002073],
 [6002244, 6000194],
 [6001209, 6000606],
 [6000352, 6001248],
 [6000549, 6001508, 6001242],
 [6001918, 6001997],
 [6001651, 6001900],
 [6000263, 6001570, 6002374],
 [6001822, 6001653],
 [6000763, 6000668],
 [6000764, 6000667],
 [6001543, 6001541],
 [6000762, 6000773],
 [6000771, 6000670],
 [6001747, 6001952],
 [6000548, 6002203],
 [6000172, 6002133],
 [6001762, 6001675],
 [6002147, 6002146],
 [6000767, 6000775],
 [6000766, 6000673],
 [6001506, 6000529],
 [6000397, 6000409],
 [6001914, 6002001],
 [6002482, 6002454],
 [6000765, 6000783],
 [6000761, 6000782],
 [6000814, 6000813],
 [6001224, 6000140]]

In [114]:
originalSongId

6000764

In [115]:
df = pd.read_csv('./data/unified/05-allDataContinousCategoricalTransition.csv')
df = df.iloc[:, 1:]
encodedFeaturesNames = pd.read_csv('./data/unified/05-encodedFeatureNames.csv')
encodedFeaturesNames = encodedFeaturesNames['0'].values.tolist()
encodedFeaturesNames.append('id')


In [116]:
df.columns

Index(['index', 'id', 'danceability', 'energy', 'speechiness', 'acousticness',
       'liveness', 'valence', 'tempo', '0',
       ...
       'mood_5', 'mood_6', 'mood_7', 'mood_8', 'mood_9', 'mood_10', 'mood_11',
       'instrumentalness_cat_0', 'instrumentalness_cat_1',
       'instrumentalness_cat_2'],
      dtype='object', length=1076)

In [117]:
dfE = df[encodedFeaturesNames]
dfE = dfE.sort_values('id')
ids = dfE.id
dfE.drop(['id'], axis=1, inplace=True)
dfE.reset_index(drop=True, inplace=True)

In [118]:
dfE = removeConstantFeatures(dfE)

In [119]:
originalSongIndex,originalSongVector = getIndexAndVector(originalSongId, ids, dfE)
coverIndex,coverSongVector = getIndexAndVector(coverSongId, ids, dfE)

In [120]:
##[0,1,1], [1,0,1] -> 0.66 Diff -> 0.33 Similarity
ranksJaccard = applyJaccard(dfE, ids, originalSongVector)

In [121]:
ranksJaccard

,id,rank
0,6000764,0.000000
1,6000998,0.333333
2,1196962,0.333333
3,6001556,0.333333
4,819426,0.333333
...,...,...
5972,739218,1.000000
5973,6001034,1.000000
5974,6001037,1.000000
5975,2107345,1.000000


In [122]:
ranksJaccard['rank'].value_counts().sort_index()

0.000000       1
0.333333     114
0.571429    1439
0.750000    3175
0.888889    1183
1.000000      65
Name: rank, dtype: int64

## Continous features

In [123]:
encodedFeaturesNames.remove('id')
df = df[df.columns.difference(encodedFeaturesNames)]
df = df.sort_values('id')
ids = df.id
df.drop(['id'], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)

## Pipeline

In [124]:
df = removeConstantFeatures(df)
df = centerAndScale(df)

mean:  [ 0. -0. -0. ... -0.  0.  0.]
std:  [1. 1. 1. ... 1. 1. 1.]


### Original Song & Cover Song

In [125]:
originalSongIndex,originalSongVector = getIndexAndVector(originalSongId, ids, df)
coverIndex,coverSongVector = getIndexAndVector(coverSongId, ids, df)

In [126]:
ranksDF  = applyEuclideanDistance(df, ids, originalSongVector)

distance


In [127]:
ranksDF.head(10)

,id,rank
0,6000764,0.000000
1,654588,4.639199
2,1227774,4.739717
3,2831476,5.107172
4,1689019,5.350137
5,6002132,5.381024
6,6000132,5.497550
7,6000717,5.506707
8,1506897,5.525174
9,6000709,5.544457


In [128]:
ranksDF[ranksDF['id']==coverSongId]

,id,rank
2087,6000667,11.355689


In [129]:
ranksDF[ranksDF['id'].isin(listCovers)]

,id,rank
0,6000764,0.000000
2087,6000667,11.355689


## Weighted rank

In [130]:
ranks = pd.merge(ranksJaccard,ranksDF, on='id')
ranks['new_rank'] = ranks['rank_x'] + ranks['rank_y']
ranks = ranks.sort_values(by='new_rank')
ranks.reset_index(drop=True, inplace=True)


In [131]:
ranks.head(10)

,id,rank_x,rank_y,new_rank
0,6000764,0.000000,0.000000,0.000000
1,654588,0.571429,4.639199,5.210627
2,1227774,0.750000,4.739717,5.489717
3,2831476,0.750000,5.107172,5.857172
4,6000171,0.333333,5.572806,5.906139
5,1689019,0.571429,5.350137,5.921565
6,6000717,0.571429,5.506707,6.078136
7,2111259,0.333333,5.768731,6.102065
8,6000709,0.571429,5.544457,6.115886
9,6002132,0.750000,5.381024,6.131024


In [132]:
ranks[ranks['id']==coverSongId]

,id,rank_x,rank_y,new_rank
2057,6000667,0.571429,11.355689,11.927118


In [133]:
ranks.loc[ranks.id.isin(listCovers)]

,id,rank_x,rank_y,new_rank
0,6000764,0.000000,0.000000,0.000000
2057,6000667,0.571429,11.355689,11.927118
